In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("result.txt", header=None)
df.columns = ['model_type', 'model_name', 'num_segments', 'batch_size', 'comments', 'time_per_batch', 'time_per_sample', 'device']
df.head()

,model_type,model_name,num_segments,batch_size,comments,time_per_batch,time_per_sample,device
0,PyTorch,TSN_r50,32,1,fp16 False cudnn False fuse_conv_bn False,52,52,1080ti
1,PyTorch,TSM_MobileNetV2,32,1,fp16 False cudnn False fuse_conv_bn False,23,23,1080ti
2,PyTorch,TSM_r50,32,1,fp16 False cudnn False fuse_conv_bn False,59,59,1080ti
3,PyTorch,TIN_r50,32,1,fp16 False cudnn False fuse_conv_bn False,141,141,1080ti
4,PyTorch,TANet,32,1,fp16 False cudnn False fuse_conv_bn False,64,64,1080ti


In [3]:
# 输出 PyTorch 的结果
# 获取 32f/16f/8f 的各类模型在 v100/1080ti/agx 上的推理速度
pytorch_df = df[df.model_type == 'PyTorch']
devices = ['v100', '1080ti', 'agx']
frames = [32, 16, 8]

def _summary(model_df):
    results = {}
    for device in devices:
        results[device] = [-1] * len(frames)
    
    for row in model_df.iterrows():
        model_name = row[0]

        cur_num_segments = row[1].num_segments
        cur_device = row[1].device
        cur_time = row[1].time_per_sample

        results[cur_device][frames.index(cur_num_segments)] = cur_time

    cur_row = []
    for device in devices:
        cur_results = [str(i) for i in results[device]]
        cur_row.append("/".join(cur_results))
    return pd.Series(cur_row)

results_by_device = pytorch_df.groupby("model_name").apply(_summary)
results_by_device.columns = ['v100(32f, 16f, 8f)', '1080ti(32f, 16f, 8f)', 'agx(32f, 16f, 8f)']
results_by_device.head()
# results_by_device.to_csv('test.csv', sep='|')

,"v100(32f, 16f, 8f)","1080ti(32f, 16f, 8f)","agx(32f, 16f, 8f)"
model_name,,,
2Plus1d_r34,61/41/32,77/41/27,539/278/146
CSN_r152,172/169/169,115/92/88,584/303/163
I3D_r50,27/23/21,21/14/11,128/68/37
SlowFast_r50,34/28/28,28/18/14,150/81/45
SlowOnly_r50,58/38/28,79/41/23,576/301/160
